In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
# !cp drive/MyDrive/rps333.zip .
# !unzip rps333

In [152]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *

In [153]:
def plot_result(history):
  loss = history.history['loss']
  accuracy = history.history['accuracy']
  epochs = len(loss)
  plt.figure(figsize=(10,5))
  plt.subplot(1,2,1)
  plt.plot(range(1,epochs+1), loss, label='train') 
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.xlim(0,epochs)
  plt.ylim(0)
  plt.grid()
  plt.subplot(1,2,2)
  plt.plot(range(1,epochs+1), accuracy, label='train') 
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.xlim(0,epochs)
  plt.ylim(0,1)
  plt.grid()
  plt.tight_layout()
  plt.savefig('loss-accuracy.png')

In [154]:
dir_train, dir_test = 'train', 'test'
drive = 'drive/MyDrive'
shape = (32,32,3)
B = 256

In [155]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255, rotation_range= 20,width_shift_range=0.4)
ds_train = data_gen.flow_from_directory(dir_train, shuffle=True, seed=0, target_size=shape[:2], batch_size=B, class_mode='sparse')

data_gen = tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255,  rotation_range= 20,width_shift_range=0.4)
ds_test = data_gen.flow_from_directory(dir_test, shuffle=True, seed=0, target_size=shape[:2], batch_size=B, class_mode='sparse')

Found 979 images belonging to 3 classes.
Found 942 images belonging to 3 classes.


In [156]:
def make_model(shape):
    model = tf.keras.models.Sequential([
    Conv2D(32,(3,3),padding='same', input_shape=shape, activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(64,(3,3),padding='same', input_shape=shape, activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(128,(3,3),padding='same', input_shape=shape, activation='relu'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(128,(3,3),padding='same', input_shape=shape, activation='relu'),

    Flatten(),

    Dense(3, activation='softmax'),




# Conv2D(64, (3,3), padding='same',
# input_shape=(32,32,3), activation='relu'),
# Conv2D(64,(3,3), padding='same', activation='relu'),
# MaxPool2D(pool_size=(2,2)),
# Dropout(0.25),
# Conv2D(128, (3,3), padding='same', activation='relu'),
# Conv2D(128, (3,3), padding='same', activation='relu'),
# MaxPool2D(pool_size=(2,2)),
# Dropout(0.25),
# Flatten(),
# Dense(512, activation='relu'),
# Dropout(0.5),
# Dense(10, activation='softmax')

    
    ])
 
    return model

In [ ]:
model = make_model(shape) 
model.compile(optimizer='adam', metrics=['accuracy'],
loss='sparse_categorical_crossentropy')

history = model.fit(ds_train, steps_per_epoch=ds_train.samples//B, epochs=64)
plot_result(history)

model.save(f'{drive}/rps.h5')

loss, accuracy = model.evaluate(ds_test)
print(f'TEST: loss:{loss:.4f} accuracy:{accuracy:.4f}')

# TEST: loss:2.7661 accuracy:0.6369

Epoch 1/64
1/3 [=========>....................] - ETA: 6s - loss: 1.1284 - accuracy: 0.2734